In [26]:
import pandas as pd
import numpy as np
import tensorflow as tf

from keras import optimizers, losses, activations, models
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from keras.layers import Dense, Input, Dropout, Convolution1D, MaxPool1D, GlobalMaxPool1D, GlobalAveragePooling1D, \
    concatenate
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split


In [27]:
df_1 = pd.read_csv("C:/00ETH/ml4h/Project1/archive/ptbdb_normal.csv", header=None)
df_2 = pd.read_csv("C:/00ETH/ml4h/Project1/archive/ptbdb_abnormal.csv", header=None)
df = pd.concat([df_1, df_2])

df_train, df_test = train_test_split(df, test_size=0.2, random_state=1337, stratify=df[187])


Y = np.array(df_train[187].values).astype(np.int8)
X = np.array(df_train[list(range(187))].values)[..., np.newaxis]

Y_test = np.array(df_test[187].values).astype(np.int8)
X_test = np.array(df_test[list(range(187))].values)[..., np.newaxis]

In [28]:
def get_model():
    nclass = 1
    inp = Input(shape=(187, 1))
    img_1 = Convolution1D(32, kernel_size=5, activation=activations.relu, padding="valid")(inp)
    img_1 = Convolution1D(32, kernel_size=5, activation=activations.relu, padding="valid")(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = Dropout(rate=0.1)(img_1)
    img_1 = Convolution1D(64, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = Convolution1D(64, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = Dropout(rate=0.1)(img_1)
    img_1 = Convolution1D(64, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = Convolution1D(64, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = Dropout(rate=0.1)(img_1)
    img_1 = Convolution1D(256, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = Convolution1D(256, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = GlobalMaxPool1D()(img_1)
    img_1 = Dropout(rate=0.2)(img_1)

    dense_1 = Dense(64, activation=activations.relu, name="dense_1")(img_1)
    dense_1 = Dense(64, activation=activations.relu, name="dense_2")(dense_1)
    dense_1 = Dense(nclass, activation=activations.sigmoid, name="dense_3_ptbdb")(dense_1)

    model = models.Model(inputs=inp, outputs=dense_1)
    opt = optimizers.Adam(0.001)

    model.compile(optimizer=opt, loss=losses.binary_crossentropy, metrics=['acc'])
    model.summary()
    return model

In [34]:
def auroc(Y_test, pred_test):
    score = tf.py_func(roc_auc_score,(Y_test, pred_test), tf.double)
    return score

In [24]:
model = get_model()
file_path = "baseline_cnn_ptbdb.h5"
checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="val_acc", mode="max", patience=5, verbose=1)
redonplat = ReduceLROnPlateau(monitor="val_acc", mode="max", patience=3, verbose=2)
callbacks_list = [checkpoint, early, redonplat]  # early

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 187, 1)            0         
_________________________________________________________________
conv1d_49 (Conv1D)           (None, 183, 32)           192       
_________________________________________________________________
conv1d_50 (Conv1D)           (None, 179, 32)           5152      
_________________________________________________________________
max_pooling1d_21 (MaxPooling (None, 89, 32)            0         
_________________________________________________________________
dropout_24 (Dropout)         (None, 89, 32)            0         
_________________________________________________________________
conv1d_51 (Conv1D)           (None, 87, 64)            6208      
_________________________________________________________________
conv1d_52 (Conv1D)           (None, 85, 64)            12352     
__________

In [18]:
model.fit(X, Y, epochs=1000, verbose=2, callbacks=callbacks_list, validation_split=0.1)
model.load_weights(file_path)

pred_test = model.predict(X_test)
pred_test = (pred_test>0.5).astype(np.int8)

Train on 10476 samples, validate on 1165 samples
Epoch 1/1000
 - 7s - loss: 0.4951 - acc: 0.7513 - val_loss: 0.4013 - val_acc: 0.8077

Epoch 00001: val_acc improved from -inf to 0.80773, saving model to baseline_cnn_ptbdb.h5
Epoch 2/1000
 - 6s - loss: 0.3277 - acc: 0.8615 - val_loss: 0.3310 - val_acc: 0.8421

Epoch 00002: val_acc improved from 0.80773 to 0.84206, saving model to baseline_cnn_ptbdb.h5
Epoch 3/1000
 - 7s - loss: 0.2621 - acc: 0.8958 - val_loss: 0.2172 - val_acc: 0.9167

Epoch 00003: val_acc improved from 0.84206 to 0.91674, saving model to baseline_cnn_ptbdb.h5
Epoch 4/1000
 - 7s - loss: 0.2191 - acc: 0.9130 - val_loss: 0.2055 - val_acc: 0.9142

Epoch 00004: val_acc did not improve from 0.91674
Epoch 5/1000
 - 7s - loss: 0.1751 - acc: 0.9316 - val_loss: 0.1366 - val_acc: 0.9382

Epoch 00005: val_acc improved from 0.91674 to 0.93820, saving model to baseline_cnn_ptbdb.h5
Epoch 6/1000
 - 7s - loss: 0.1421 - acc: 0.9464 - val_loss: 0.1206 - val_acc: 0.9579

Epoch 00006: val

In [31]:

f1 = f1_score(Y_test, pred_test)

print("Test f1 score : %s "% f1)

acc = accuracy_score(Y_test, pred_test)

print("Test accuracy score : %s "% acc)


Test f1 score : 0.997384066587396 
Test accuracy score : 0.9962212298179319 
Test accuracy score : Tensor("PyFunc_1:0", dtype=float64) 


In [37]:
model.compile(loss='binary_crossentropy', optimizer = 'adam', metrics=['accuracy',auroc])
model.fit(X, Y, epochs=1000, verbose=2, callbacks=callbacks_list, validation_split=0.1)

Train on 10476 samples, validate on 1165 samples
Epoch 1/1000
 - 7s - loss: 0.5026 - acc: 0.7410 - auroc: 0.7691 - val_loss: 0.4115 - val_acc: 0.8283 - val_auroc: 0.8750

Epoch 00001: val_acc improved from -inf to 0.82833, saving model to baseline_cnn_ptbdb.h5
Epoch 2/1000
 - 6s - loss: 0.3342 - acc: 0.8627 - auroc: 0.9192 - val_loss: 0.2533 - val_acc: 0.9013 - val_auroc: 0.9602

Epoch 00002: val_acc improved from 0.82833 to 0.90129, saving model to baseline_cnn_ptbdb.h5
Epoch 3/1000
 - 6s - loss: 0.2332 - acc: 0.9079 - auroc: 0.9596 - val_loss: 0.1685 - val_acc: 0.9425 - val_auroc: 0.9730

Epoch 00003: val_acc improved from 0.90129 to 0.94249, saving model to baseline_cnn_ptbdb.h5
Epoch 4/1000
 - 6s - loss: 0.1895 - acc: 0.9284 - auroc: 0.9758 - val_loss: 0.1645 - val_acc: 0.9433 - val_auroc: 0.9821

Epoch 00004: val_acc improved from 0.94249 to 0.94335, saving model to baseline_cnn_ptbdb.h5
Epoch 5/1000
 - 6s - loss: 0.1521 - acc: 0.9443 - auroc: 0.9824 - val_loss: 0.1349 - val_acc: 

InvalidArgumentError: ValueError: Only one class present in y_true. ROC AUC score is not defined in that case.
Traceback (most recent call last):

  File "C:\Users\user\Anaconda\envs\ml4h\lib\site-packages\tensorflow\python\ops\script_ops.py", line 209, in __call__
    ret = func(*args)

  File "C:\Users\user\Anaconda\envs\ml4h\lib\site-packages\sklearn\metrics\_ranking.py", line 572, in roc_auc_score
    sample_weight=sample_weight,

  File "C:\Users\user\Anaconda\envs\ml4h\lib\site-packages\sklearn\metrics\_base.py", line 75, in _average_binary_score
    return binary_metric(y_true, y_score, sample_weight=sample_weight)

  File "C:\Users\user\Anaconda\envs\ml4h\lib\site-packages\sklearn\metrics\_ranking.py", line 338, in _binary_roc_auc_score
    "Only one class present in y_true. ROC AUC score "

ValueError: Only one class present in y_true. ROC AUC score is not defined in that case.


	 [[{{node metrics_8/auroc/PyFunc}}]]